In [ ]:
import os
from pathlib import Path
import numpy as np

from applied_planning.envs import Lite6CubePointingEnv

BASE_PATH = "/Users/braeden/Development/applied-planning/"
model_path = BASE_PATH + "src/applied_planning/sim/assets/ufactory_lite6/lite6_gripper_narrow.xml"

# create environment
print("Creating environment...")
env = Lite6CubePointingEnv(
    model_path=str(model_path),
    render_mode=None,           # set to "human" to watch training (will be slow!)
    max_steps=200,
    success_threshold=0.05,     # 5cm
    reward_scale=10.0,
    cube_placement_radius=0.3,
    ee_site_name="end_effector"
)

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

Creating environment...
✓ Added 1 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)
Observation space: Box(-inf, inf, (12,), float32)
Action space: Box(-1.0, 1.0, (6,), float32)


In [ ]:
obs, info = env.reset()
print(f"Initial observation shape: {obs.shape}")
print(f"Cube position: {info['cube_pos']}")
print(f"EE position: {info['ee_pos']}")
print(f"Initial distance: {info['distance']:.4f} m")

# run a few random steps
for step in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {step+1}: distance={info['distance']:.4f} m, reward={reward:.2f}")
    if terminated or truncated:
        break

Initial observation shape: (12,)
Cube position: [0.02319782 0.15650207 0.03      ]
EE position: [ 8.69984251e-02 -2.11781530e-06  7.24896309e-02]
Initial distance: 0.1743 m
Step 1: distance=0.6744 m, reward=-16.75
Step 2: distance=0.1393 m, reward=9.31
Step 3: distance=0.3536 m, reward=-7.82
Step 4: distance=0.4432 m, reward=-6.22
Step 5: distance=0.4873 m, reward=-5.75
Step 6: distance=0.6947 m, reward=-11.10
Step 7: distance=0.6518 m, reward=-5.66
Step 8: distance=0.8198 m, reward=-11.56
Step 9: distance=0.6649 m, reward=-3.55
Step 10: distance=0.2001 m, reward=7.29


In [3]:
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

# make environemnt for stable-baselines3
def make_env():
    return Lite6CubePointingEnv(
        model_path=str(model_path),
        render_mode=None,
        max_steps=200,
        success_threshold=0.05,
        reward_scale=10.0,
        cube_placement_radius=0.3,
        ee_site_name="end_effector"
    )

# create vectorized environment
train_env = DummyVecEnv([make_env])
eval_env = DummyVecEnv([make_env])

# setup callbacks
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/best_model",
    log_path="./logs/eval",
    eval_freq=5000,
    deterministic=True,
    render=False
)

checkpoint_callback = CheckpointCallback(
    save_freq=10000,
    save_path="./logs/checkpoints",
    name_prefix="cube_pointing_model"
)

# create RL model - using SAC (works well for continuous control)
print("\nCreating SAC model...")
model = SAC(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    buffer_size=100000,
    learning_starts=1000,
    batch_size=256,
    tau=0.005,
    gamma=0.99,
    train_freq=1,
    gradient_steps=1,
    tensorboard_log="./logs/tensorboard"
)

print("\n" + "="*60)
print("Training Configuration:")
print("="*60)
print(f"Algorithm: SAC")
print(f"Total timesteps: 100,000")
print(f"Eval frequency: 5,000 steps")
print(f"Checkpoint frequency: 10,000 steps")
print("="*60)

# train the model
print("\nStarting training...")
print("(This will take a while. Press Ctrl+C to stop early)\n")

try:
    model.learn(
        total_timesteps=100000,
        callback=[eval_callback, checkpoint_callback],
        progress_bar=True
    )

    # save final model
    model.save("./logs/cube_pointing_final")
    print("\n✓ Training complete! Model saved to ./logs/cube_pointing_final")

except KeyboardInterrupt:
    print("\n\nTraining interrupted by user")
    model.save("./logs/cube_pointing_interrupted")
    print("Model saved to ./logs/cube_pointing_interrupted")



✓ Added 1 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)
✓ Added 1 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)

Creating SAC model...
Using cpu device

Training Configuration:
Algorithm: SAC
Total timesteps: 100,000
Eval frequency: 5,000 steps
Checkpoint frequency: 10,000 steps

Starting training...
(This will take a while. Press Ctrl+C to stop early)

Logging to ./logs/tensorboard/SAC_2


Output()

---------------------------------
| rollout/           |          |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2398     |
|    time_elapsed    | 0        |
|    total_timesteps | 800      |
---------------------------------


---------------------------------
| rollout/           |          |
|    success_rate    | 0.125    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 435      |
|    time_elapsed    | 3        |
|    total_timesteps | 1549     |
| train/             |          |
|    actor_loss      | 17.1     |
|    critic_loss     | 31       |
|    ent_coef        | 0.851    |
|    ent_coef_loss   | -1.48    |
|    learning_rate   | 0.0003   |
|    n_updates       | 548      |
---------------------------------


---------------------------------
| rollout/           |          |
|    success_rate    | 0.167    |
| time/              |          |
|    episodes        | 12       |
|    fps             | 309      |
|    time_elapsed    | 7        |
|    total_timesteps | 2178     |
| train/             |          |
|    actor_loss      | 48.3     |
|    critic_loss     | 19.1     |
|    ent_coef        | 0.714    |
|    ent_coef_loss   | -2.88    |
|    learning_rate   | 0.0003   |
|    n_updates       | 1177     |
---------------------------------


...

Eval num_timesteps=100000, episode_reward=-497.13 +/- 63.25

Episode length: 200.00 +/- 0.00

Success rate: 0.00%

---------------------------------
| eval/              |          |
|    mean_ep_length  | 200      |
|    mean_reward     | -497     |
|    success_rate    | 0        |
| time/              |          |
|    total_timesteps | 100000   |
| train/             |          |
|    actor_loss      | 389      |
|    critic_loss     | 83.3     |
|    ent_coef        | 0.164    |
|    ent_coef_loss   | 0.998    |
|    learning_rate   | 0.0003   |
|    n_updates       | 98999    |
---------------------------------



✓ Training complete! Model saved to ./logs/cube_pointing_final


In [4]:
# Test trained model
print("\n" + "="*60)
print("Testing trained model...")
print("="*60)

test_env = Lite6CubePointingEnv(
    model_path=str(model_path),
    render_mode="human",  # Render to see the trained agent
    max_steps=200,
    success_threshold=0.05,
    reward_scale=10.0,
    cube_placement_radius=0.3,
    ee_site_name="end_effector"
)

for episode in range(3):
    obs, info = test_env.reset()
    print(f"\nEpisode {episode+1} - Initial distance: {info['distance']:.4f} m")

    episode_reward = 0
    for step in range(200):
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = test_env.step(action)
        episode_reward += reward

        if terminated or truncated:
            print(f"  Finished in {step+1} steps")
            print(f"  Final distance: {info['distance']:.4f} m")
            print(f"  Success: {info['is_success']}")
            print(f"  Total reward: {episode_reward:.2f}")
            break

test_env.close()


Testing trained model...
✓ Added 1 cubes to scene within 0.3m radius
IK solver using site: end_effector
⚠ Viewer could not be launched: `launch_passive` requires that the Python script be run under `mjpython` on macOS
  Running in headless mode
  Tip: Use render_mode='offscreen' for notebook visualization

Episode 1 - Initial distance: 0.1615 m
  Finished in 200 steps
  Final distance: 0.2346 m
  Success: False
  Total reward: -470.67

Episode 2 - Initial distance: 0.1990 m
  Finished in 200 steps
  Final distance: 0.1921 m
  Success: False
  Total reward: -447.75

Episode 3 - Initial distance: 0.2178 m
  Finished in 200 steps
  Final distance: 0.3671 m
  Success: False
  Total reward: -594.53


In [5]:
env.close()

In [6]:
# Load and test the trained model in MuJoCo viewer
import time
from stable_baselines3 import SAC

print("="*60)
print("Testing Trained Model in MuJoCo Viewer")
print("="*60)

# Load the best model (or final model if you prefer)
try:
    model = SAC.load("./logs/best_model/best_model")
    print("✓ Loaded best model from evaluation")
except:
    model = SAC.load("./logs/cube_pointing_final")
    print("✓ Loaded final model")

# Create environment with rendering enabled
test_env = Lite6CubePointingEnv(
    model_path=str(model_path),
    render_mode="human",  # This will open MuJoCo viewer
    max_steps=200,
    success_threshold=0.05,
    reward_scale=10.0,
    cube_placement_radius=0.3,
    ee_site_name="end_effector"
)

print("\nRunning 5 test episodes...")
print("(MuJoCo viewer window should open)\n")

success_count = 0
total_rewards = []

for episode in range(5):
    obs, info = test_env.reset()
    print(f"\nEpisode {episode+1}:")
    print(f"  Cube position: {info['cube_pos']}")
    print(f"  Initial distance: {info['distance']:.4f} m")
    
    episode_reward = 0
    done = False
    
    for step in range(200):
        # Get action from trained model
        action, _states = model.predict(obs, deterministic=True)
        
        # Step environment
        obs, reward, terminated, truncated, info = test_env.step(action)
        episode_reward += reward
        
        # Small delay so we can watch the movement
        time.sleep(0.02)
        
        if terminated or truncated:
            done = True
            break
    
    # Print episode results
    print(f"  Steps taken: {step+1}")
    print(f"  Final distance: {info['distance']:.4f} m")
    print(f"  Success: {info['is_success']}")
    print(f"  Total reward: {episode_reward:.2f}")
    
    if info['is_success']:
        success_count += 1
    total_rewards.append(episode_reward)

test_env.close()

print("\n" + "="*60)
print("Test Summary:")
print("="*60)
print(f"Success rate: {success_count}/5 ({success_count/5*100:.1f}%)")
print(f"Average reward: {np.mean(total_rewards):.2f} ± {np.std(total_rewards):.2f}")
print("="*60)

Testing Trained Model in MuJoCo Viewer
✓ Loaded best model from evaluation
✓ Added 1 cubes to scene within 0.3m radius
IK solver using site: end_effector

Running 5 test episodes...
(MuJoCo viewer window should open)

⚠ Viewer could not be launched: `launch_passive` requires that the Python script be run under `mjpython` on macOS
  Running in headless mode
  Tip: Use render_mode='offscreen' for notebook visualization

Episode 1:
  Cube position: [0.08102471 0.18027519 0.03      ]
  Initial distance: 0.1853 m
  Steps taken: 200
  Final distance: 0.1637 m
  Success: False
  Total reward: -306.67

Episode 2:
  Cube position: [0.15893276 0.0362336  0.03      ]
  Initial distance: 0.0911 m
  Steps taken: 200
  Final distance: 0.0760 m
  Success: False
  Total reward: -351.45

Episode 3:
  Cube position: [ 0.25480012 -0.01995478  0.03      ]
  Initial distance: 0.1742 m
  Steps taken: 200
  Final distance: 0.2640 m
  Success: False
  Total reward: -468.18

Episode 4:
  Cube position: [-0.157